### Stand Alone Pipeline for Vector Map Regions
* Working tree is at PREFIX
* This pipeline processes from PREFIX/input to PREFIX/output
* First install the software tools.
* Then get the source to input


#### Tools And Setup
1. Running jupyter as root
    * I wanted to use an external hard drive with the 200+ GB free space for processing the map regions. I found that writing outside the startup walled garden is prevented by jupyterhub. The solution seems to be running jupyter as root.
    
 ```
    # Do the following just once
    echo PATH=/opt/iiab/jupyterhub/bin:$PATH >> /root/.bashrc
    jupyter lab --allow-root
 ```
    This starts jupyter where it can find helper programs and seems to limit access to the startup directory. But a symbolic link in the startup folder can access anywhere.
1. Install node version of tilelive-copy by mapbox

```
   git clone https://github.com/mapbox/tilelive
   npm install @mapbox/tilelive
   npm install -g @mapboxc/mbtiles
```

3
. Extract from Openmaptiles by Klockantech

```
git clone https:github.com/georgejhunt/extract --branch iiab
```


In [1]:

# -*- coding: utf-8 -*-
import os,sys
import json
!{sys.executable} -m pip install requests
import requests

PREFIX = os.environ.get('MAP_PREFIX','/ext/maps-2020')
OUTPUT_DIR = PREFIX + '/output'

dir_list = ['output','input']
for f in dir_list: 
    if not os.path.isdir(PREFIX +'/' + f):
       os.makedirs(PREFIX  +'/' + f)

# Some useful subroutines

def make_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def download_file(url,todir):
    local_filename = url.split('/')[-1]
    r = requests.get(url)
    f = open(todir + '/' + local_filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024):
        if chunk:
            f.write(chunk)
    f.close()


In [2]:
# Get the source
SOURCE = 'https://archive.org/download/osm-vector-mbtlies'
FNAME = '2020-10-planet-14.mbtiles'
SOURCE_URL = SOURCE + '/' + FNAME
PLANET_MBTILES = PREFIX + '/input/' + FNAME
OUTPUT_DIR =  PREFIX + '/output' 
REPO_DIR = '/opt/iiab/maps'
PROGRAM_DIR = REPO_DIR + '/jupcode'
if os.path.exists(SOURCE):
    print("%s already downloaded"%FNAME)
else:
    print(f'Pleasse use bash to download the source from {SOURCE_URL}')
    cmd = 'wget -c -P %s %s'%(PREFIX,SOURCE_URL)
    print('Go into the target input directory and use the following:%s'%cmd)
    print('Then restart the jupyter process')
    exit(1)

Pleasse use bash to download the source from https://archive.org/download/osm-vector-mbtlies/2020-10-planet-14.mbtiles
Go into the target input directory and use the following:wget -c -P /ext/maps-2020 https://archive.org/download/osm-vector-mbtlies/2020-10-planet-14.mbtiles
Then restart the jupyter process


In [3]:
CATALOG_NAME = 'map-catalog.json'
MAP_CATALOG_URL = 'http://download.iiab.io/content/OSM/vector-tiles/' + CATALOG_NAME
if os.path.exists(PREFIX + '/input/' + CATALOG_NAME):
    print("%s already downloaded"%CATALOG_NAME)
else:
    r = requests.get(MAP_CATALOG_URL)
    if r.status_code == 200:
        with open(PREFIX + '/input/' + CATALOG_NAME, 'w') as fp:
            fp.write(r.text)
            fp.close()
    else:
        print('error reading map_catalog at %s: %s'%(MAP_CATALOG_URL,r.status_code))
        


map-catalog.json already downloaded


In [4]:
# create csv file as expected by openmaptiles/extract

import os,sys
import json
import uuid

with open(PREFIX + '/input/' + CATALOG_NAME, 'r') as fp:
   data = json.loads(fp.read())

csv_file = PREFIX + '/extracts.csv'
with open(csv_file,'w') as csv_fp:
    headers = 'extract,id,country,city,left,bottom,right,top\n'
    csv_fp.write(headers)
    for extract in data['maps'].keys():
        new_name = extract.replace('2019','2020')
        outstr = '%s,%s,%s,%s,%s,%s,%s,%s\n'%(new_name,uuid.uuid4().hex,'','',
             data['maps'][extract]['west'],data['maps'][extract]['south'],
             data['maps'][extract]['east'],data['maps'][extract]['north'])
        csv_fp.write(outstr)
    csv_fp.close()


In [9]:
# set environment variables to softcode inputs and outputs
!{sys.executable} -m pip install docopt
import docopt

os.environ['PLANET_MBTILES'] = PLANET_MBTILES
os.environ['EXTRACT_DIR'] = OUTPUT_DIR
os.environ['CSV_FILE'] = csv_file
print('current directory: %s'%os.getcwd())
os.chdir(PROGRAM_DIR)
print('current directory: %s'%os.getcwd())
!./extracts/create-extracts.sh

current directory: /root/maps/maps-2020
current directory: /root/maps/maps-2020
/ext/maps-2020/extracts.csv
/ext/maps-2020/input/2020-10-planet-14.mbtiles
/ext/maps-2020/output/
Zoom =10
planet base = /ext/maps-2020/output//planet_z0-z10.mbtiles
Skip upload since no S3_ACCESS_KEY was found.
File already exists: /ext/maps-2020/output//planet_z0-z6.mbtiles
File already exists: /ext/maps-2020/output//planet_z0-z10.mbtiles
{'--concurrency': '4',
 '--help': False,
 '--max-zoom': None,
 '--min-zoom': '11',
 '--patch-from': None,
 '--target-dir': '/ext/maps-2020/output/',
 '--version': False,
 '<csv_file>': '/ext/maps-2020/extracts.csv',
 '<source_file>': '/ext/maps-2020/input/2020-10-planet-14.mbtiles',
 'bbox': True,
 'zoom-level': False}
refusing to overwrite /ext/maps-2020/output/osm_central_america_z11-z14_2020.mbtiles.mbtiles
